This is an question-answering system project for academic papers. In this project, Pre-trained Distilbert is used from Huggingface and it is finetuned.

The **best model**'s pt is located on https://drive.google.com/uc?export=download&confirm=1Eah&id=1_w8AIn1ET-xbk8SAyrd_qYTgjBxMAqZP

# Installations

In order to mouth to the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_directory = '/content/model_directory'
!mkdir model_directory
model_directory_for_drive = '/content/drive/MyDrive/model_directory_for_drive'
!mkdir '/content/drive/MyDrive/model_directory_for_drive'

mkdir: cannot create directory ‘model_directory’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/model_directory_for_drive’: File exists


In this project, huggingface library is used; in order to use them with GoogleColab, we need to first install them with pip command

In [ ]:
!pip install transformers
!pip install datasets
!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
!pip install pdftotext

     |████████████████████████████████| 2.5MB 5.1MB/s 
     |████████████████████████████████| 901kB 28.9MB/s 
     |████████████████████████████████| 3.3MB 40.5MB/s 
     |████████████████████████████████| 245kB 5.6MB/s 
     |████████████████████████████████| 245kB 43.0MB/s 
     |████████████████████████████████| 122kB 51.1MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.7 kB of archives.
After this operation, 188 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.

# Impoting Files

In [ ]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, AdamW
from torch.utils.data import DataLoader, Dataset
import torch as nn
import os
import gdown # in order to download from google drive
import time # in order to count the time
import pdftotext # in order to convert PDF to string
import re # in order to remove unnecessary spaces in the string

# Checking Available Device

In [ ]:
device = nn.device('cuda') if nn.cuda.is_available() else nn.device('cpu')
if nn.cuda.is_available():
  print(f"Available Device is GPU.")
else:
  print(f"Available Device is CPU.")

Available Device is GPU.


# Loading Dataset

In this project, Squad dataset is used from Standford University

In [ ]:
dataset = load_dataset('squad')

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


Visualizing the Dataset and Its Dictionary Type

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 10570
    })
})


Creating each list from dataset 

In [ ]:
train_contexts = dataset['train']['context']
train_questions = dataset['train']['question']
train_answers = dataset['train']['answers']
val_contexts = dataset['validation']['context']
val_questions = dataset['validation']['question']
val_answers = dataset['validation']['answers']

Convert Answer Start-Answer End-Text from List to Int-String

In [ ]:
for i in range(len(train_answers)):
  (train_answers[i]['answer_start']) = (train_answers[i]['answer_start'][0])
  (train_answers[i]['text']) = (train_answers[i]['text'][0])

for i in range(len(val_answers)):
  (val_answers[i]['answer_start']) = (val_answers[i]['answer_start'][0])
  (val_answers[i]['text']) = (val_answers[i]['text'][0])

This part adds the end idx to end of the answer, and it is taken from https://gist.github.com/jamescalam/02f63ba35c0573f2c1bd45267a4581ed

In [ ]:
# TAKEN FROM https://gist.github.com/jamescalam/02f63ba35c0573f2c1bd45267a4581ed

def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        start_idx = start_idx
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            # this means the answer is off by 1-2 tokens
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
# and apply the function to our two answer lists
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

Tokenizer

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased-distilled-squad")

Encodings for train and validation test are created with the help of Bert Tokenizer

In [ ]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
validation_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

This part of the function is taken from https://huggingface.co/transformers/custom_datasets.html 
* It is used for adding start and end positions of tokens 

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(validation_encodings, val_answers)

# Dataset Creation

In [ ]:
class MyDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: nn.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
train_dataset = MyDataset(train_encodings)
val_dataset = MyDataset(validation_encodings)

Data Loaders are created with the help of PyTorch

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

# Training

##Definition of the Model

In [ ]:
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad") 
model.to(device)
model.train()

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

##Adam Optimizer with Learning Rate = 5e-5

In [ ]:
optim = AdamW(model.parameters(), lr=5e-5)

## Training with PyTorch

I have stopped the training at the beginning of 3rd epoch because validation accuracy has started to decrease and each epoch was lasting approximately 1.5 hours.

In [ ]:
for epoch in range(3):
    print("-------------------")
    print(f"Epoch : {epoch} is started.")
    start_time = time.time()
    ############### Training Process ###############

    model.train() # In order to convert model for training process
    loss_for_every_epoch = 0  # In order to visualize losses for each epoch
    number_of_batch = 0
    for each_batch in train_loader:
        number_of_batch += 1

        optim.zero_grad()

        input_ids = each_batch['input_ids'].to(device)
        attention_mask = each_batch['attention_mask'].to(device)
        start_positions = each_batch['start_positions'].to(device)
        end_positions = each_batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        
        loss = outputs[0]

        loss.backward()
        loss_for_every_epoch = loss_for_every_epoch + loss.item()
        optim.step()

    if epoch == 0:
      training_loss1 = loss_for_every_epoch
      training_loss1 = loss_for_every_epoch/number_of_batch
      print(f"Training_Loss_For_Epoch = 1: {training_loss1}")
      epoch_0_time = time.time()- start_time 
      print(f"Epoch 0 - is lasted {epoch_0_time} seconds")
    elif epoch == 1:
      training_loss2 = loss_for_every_epoch
      training_loss2 = loss_for_every_epoch/number_of_batch
      print(f"Training_Loss_For_Epoch = 2: {training_loss2}")
      epoch_1_time = time.time()- start_time 
      print(f"Epoch 1 - is lasted {epoch_1_time} seconds")
    elif epoch == 2:
      training_loss3 = loss_for_every_epoch
      training_loss3 = loss_for_every_epoch/number_of_batch
      print(f"Training_Loss_For_Epoch = 3: {training_loss3}")
      epoch_2_time = time.time()- start_time 
      print(f"Epoch 2 - is lasted {epoch_2_time} seconds")


    # Saving The Model
    nn.save(model.state_dict(), os.path.join(model_directory, 'epoch-{}.pt'.format(epoch)))
    nn.save(model.state_dict(), os.path.join(model_directory_for_drive, 'epoch-{}-withlr=5e-5.pt'.format(epoch)))


    
    ############### Validation Process ###############

    accuracy = []
    model.eval()
    for each_batch in val_loader:
      with nn.no_grad():
        input_ids = each_batch['input_ids'].to(device)
        attention_mask = each_batch['attention_mask'].to(device)
        start_positions = each_batch['start_positions'].to(device)
        end_positions = each_batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        
        start_pos_prediction = nn.argmax(outputs['start_logits'], dim = 1)
        end_pos_prediction   = nn.argmax(outputs['end_logits'], dim = 1)

        # Append to accuracy list if start positions match with prediction
        accuracy.append(((start_positions == start_pos_prediction).sum()/len(start_pos_prediction)).item())
        # Append to accuracy list if end positions match with prediction
        accuracy.append(((end_positions == end_pos_prediction).sum()/len(end_pos_prediction)).item())

    if epoch == 0:
      validation_Accuracy1 = sum(accuracy)/len(accuracy)
      print(f"validation_accuracy1: {validation_Accuracy1}")
    elif epoch == 1:
      validation_Accuracy2 = sum(accuracy)/len(accuracy)
      print(f"validation_accuracy2: {validation_Accuracy2}")
    elif epoch == 2:
      validation_Accuracy3 = sum(accuracy)/len(accuracy)
      print(f"validation_accuracy3: {validation_Accuracy3}")
    

-------------------
Epoch : 0 is started.
Training_Loss_For_Epoch = 1: 1.0677542532415696
Epoch 0 - is lasted 5114.922166347504 seconds
validation_accuracy1: 0.680285552220698
-------------------
Epoch : 1 is started.
Training_Loss_For_Epoch = 2: 0.7863054304770684
Epoch 1 - is lasted 5094.442261695862 seconds
validation_accuracy2: 0.6691187594463534
-------------------
Epoch : 2 is started.


# Loading the Best Model for Evaluation

Downloading the best model from Google Drive

In [ ]:
url = 'https://drive.google.com/uc?export=download&confirm=1Eah&id=1_w8AIn1ET-xbk8SAyrd_qYTgjBxMAqZP'
output = '/content/bestmodel.pt'
gdown.download(url, output, quiet=False) 

Downloading...
From: https://drive.google.com/uc?export=download&confirm=1Eah&id=1_w8AIn1ET-xbk8SAyrd_qYTgjBxMAqZP
To: /content/bestmodel.pt
265MB [00:01, 173MB/s]


'/content/bestmodel.pt'

In [ ]:
best_model_directory = '/content/bestmodel.pt'
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad", return_dict=False)
model.load_state_dict(nn.load(best_model_directory, map_location=nn.device('cpu')))
model.eval()

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

# RESULTS

In this project, *A Review Paper on Cloud Computing. International Journal of Advanced Research in Computer Science and Software Engineering* by Srivastava, Priyanshu and Khan, Rizwan is used for testing the model


Downloading the PDF

In [ ]:
url = 'https://www.researchgate.net/profile/Rizwan-Khan-27/publication/326073288_A_Review_Paper_on_Cloud_Computing/links/5b3c841fa6fdcc8506eef9de/A-Review-Paper-on-Cloud-Computing.pdf?_sg%5B0%5D=pvwAS7i5EBemqW2sqqs6RHvE3tEt0COVJFzfFR5mMmJHC3n-tk8cwxKEBmoeB8oGK9UVlFIW8YIKawvVkQNeCg.KhuXvVUSRqj_EmEMV3FTPmDqvh5xWg4EtQdgLLvutDEdZpQ-llCT3bW8OW_0k_3Z8JulaSlwA8C8oYUE0K6Drg&_sg%5B1%5D=QFnk9OEV5HupL0QxLa7Oknx1rPG8UJugh896J_ru6cnNTpCCseUIBHEfSQycDnGoOTBn8ncoxWwQCkFLEWAMp0kXNFkvVZ2ZMD8sFxvI9Qta.KhuXvVUSRqj_EmEMV3FTPmDqvh5xWg4EtQdgLLvutDEdZpQ-llCT3bW8OW_0k_3Z8JulaSlwA8C8oYUE0K6Drg&_sg%5B2%5D=pRccRT6x9bicwA6KygPJfnMGfg3-12EZSWuGqSK5iKHYSr5S_8cv0HV75pWbQbw46OYHEwCJr_NLboI.6Q2SxdqlhYDCW--wISJjvgKk53IUvLWfI3UE20-PKeFNJ794BoGPCMA66-UbjV7zyDDmFn446H9K69KZTwTWiA&_iepl='
output = '/content/711-1636-1-PB.pdf'
gdown.download(url, output, quiet=False) 

Downloading...
From: https://www.researchgate.net/profile/Rizwan-Khan-27/publication/326073288_A_Review_Paper_on_Cloud_Computing/links/5b3c841fa6fdcc8506eef9de/A-Review-Paper-on-Cloud-Computing.pdf?_sg%5B0%5D=pvwAS7i5EBemqW2sqqs6RHvE3tEt0COVJFzfFR5mMmJHC3n-tk8cwxKEBmoeB8oGK9UVlFIW8YIKawvVkQNeCg.KhuXvVUSRqj_EmEMV3FTPmDqvh5xWg4EtQdgLLvutDEdZpQ-llCT3bW8OW_0k_3Z8JulaSlwA8C8oYUE0K6Drg&_sg%5B1%5D=QFnk9OEV5HupL0QxLa7Oknx1rPG8UJugh896J_ru6cnNTpCCseUIBHEfSQycDnGoOTBn8ncoxWwQCkFLEWAMp0kXNFkvVZ2ZMD8sFxvI9Qta.KhuXvVUSRqj_EmEMV3FTPmDqvh5xWg4EtQdgLLvutDEdZpQ-llCT3bW8OW_0k_3Z8JulaSlwA8C8oYUE0K6Drg&_sg%5B2%5D=pRccRT6x9bicwA6KygPJfnMGfg3-12EZSWuGqSK5iKHYSr5S_8cv0HV75pWbQbw46OYHEwCJr_NLboI.6Q2SxdqlhYDCW--wISJjvgKk53IUvLWfI3UE20-PKeFNJ794BoGPCMA66-UbjV7zyDDmFn446H9K69KZTwTWiA&_iepl=
To: /content/711-1636-1-PB.pdf
100%|██████████| 400k/400k [00:00<00:00, 9.73MB/s]


'/content/711-1636-1-PB.pdf'

Converting PDF to string with pdftotext lib

In [ ]:
# Load PDF file
with open("/content/711-1636-1-PB.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

Manually coverted PDF

In [ ]:
manuel_pdf = [
       # First Page
       """Abstract— Today is the era of Cloud Computing Technology in IT Industries. Cloud computing which is based on
Internet has the most powerful architecture of computation. It reckons in of a compilation of integrated and
networked hardware, software and internet infrastructure. It has various avails atop grid computing and other
computing. In this paper, I have given a brief of evaluation of cloud computing by reviewing more than 30 articles on
cloud computing. The outcome of this review signalizes the face of the IT industries before and after the cloud
computing.
Keywords— Cloud, SaaS, PaaS, IaaS, Cloud Computing.
I. INTRODUCTION
Like real clouds which are the collection of water molecules, the term ‗cloud‘ in cloud computing is the
collection of networks. The user can use the modalities of cloud computing boundlessly whenever demanded. Instead of
setting up their own physical infrastructure, the users ordinarily prefer a mediator provider for the service of the internet
in cloud computing. The users have to pay only for the services they had used [2]. The workload can be shifted to reduce
the workload in cloud computing. A load of service is handled by the networks which forms the cloud that's why the load
on local computers is not heavy while running an application [1]. So the requisition of hardware and software at the user
side is decreased. All we need to have a web browser to use cloud computing. All we need to have a web browser like
chrome to use cloud computing. Following are the key features of cloud computing:
I.I Resource Pooling and Elasticity
I.II Self-Service and On-Demand Services
I.III Pricing
I.IV Quality of Service
There are three services provided by cloud computing that are Software as a Service (SaaS), Platform as a
Service (PaaS) and Infrastructure as a Service (IaaS) [1]. The basic examples of cloud computing which are used by
general people in daily life are Facebook, YouTube, Dropbox, and Gmail etc. It offers scalability, flexibility, agility, and
simplicity that's why its use is rapidly increasing in the enterprises.
Fig 1 Network of Cloud
II. EVOLUTION OF CLOUD COMPUTING
One day in a speech at MIT around in 1960 John McCarthy indicated that like water and electricity, computing
can also be sold like a utility. And in 1999, the Salesforce Company started distributing the applications to the customers """,
       # Second Page
       """through a convenient website [3]. Amazon Web Services were started by Amazon in 2002 and they were providing the
services of storage and computation. In around 2009 big companies like Google, Microsoft, HP, Oracle had started to
provide cloud computing services [4]. Nowadays each and every person is using the services of cloud computing in their
daily life. For example Google Photos, Google Drive, and iCloud etc. In future cloud computing will become the basic
need of IT Industries.
Fig 2 Evolution of Cloud Computing
III. COMPONENTS OF CLOUD COMPUTING
Cloud computing has three basic components as followsIII.I Client Computers: The end user can interact with the cloud using the client computers.
III.II Distributed Servers: The servers are distributed among the different places but acts like they as working
with each other.
III.III Data Centres: Data centres are the compilation of servers.
Fig 3 Components
IV. SERVICES OF CLOUD COMPUTING
IV.I Software as a Service (SaaS): The way of carrying application as a service on the internet is known as
software as a service. In place of installing the software on his computer, the user can simply access it via the internet [5].""",
       # Third Page
       """It makes the user free from managing the complex software and hardware. The SaaS users do not need to buy software or
hardware, maintain, and update. The only thing user must have an internet connection and then access to the application
is very easy. Example, Microsoft Office 365, Google Apps etc.
IV.II Platform as a Service (PaaS): A development environment or platform is given to the consumers as a
service in PaaS, upon which user can deploy their own software and coding. The customer has the liberty to construct his
own applications that can run on the provider's infrastructure [5]. Product as a service providers offers a predefined
composition of operating system and application server to obtain the management capacity of the applications. For
example, LAMP (Linux, Apache, MySQL, and PHP), J2EE, Ruby etc.
IV.III Infrastructure as a Service (IaaS): Many computing resources are provided by the IaaS in the form of
storage, network, operating system, hardware, and storage devices on demand. IaaS users can access the services using a
wide area network, such as the internet [5]. For example, a user can create virtual machines by login to the IaaS platform.
Fig 4 Cloud Computing Services
V. TYPES OF CLOUD COMPUTING
V.I Public Cloud: The public cloud is a computing service supplied by the third party providers atop the public
internet [6]. These services are available for any user who wants to use them and they have to pay only for the services
they consumed.
V.II Private Cloud: The computing services provided over the internet or private network come under the
private cloud and these services are offered only to the selected users in place of common people [1,6]. A higher security
and privacy is delegated by private clouds through the firewall and internal hosting
.
V.II Hybrid Cloud: Hybrid cloud is the combination of public cloud and private cloud. In the hybrid cloud,
each cloud can be managed independently but data and applications can be shared among the clouds in the hybrid cloud
[1, 6].
VI. BENEFITS OF CLOUD COMPUTING
VI.I Cost Saving: In cloud computing users have to only pay for the services they consumed. Maintenance cost
is low as user do not need to purchase the infrastructure [2].
VI.II Flexibility: Cloud computing is scalable. The rapid scale up and down in the operations of your business
may require quick adjustment of hardware and resources so in order to manage this variations cloud computing provide
flexibility.
VI.III Enhanced Security: Cloud computing provide high security by using the data encryption, strong access
controls, key management, and security intelligence.""",
       # Forth Page
       """In this review paper we described in short the introduction, evolution, types and components of cloud computing
and also different approaches of cloud computing and some of its advantages. The application area of cloud computing
will continuously be increasing. Today approximately all small and big industries are using cloud computing to manage
storage, traffic, hardware requirements. So, it is clear that there is major impact of cloud computing on society and
business.
ACKNOWLEDGMENT
We are thankful to Head of Department Mr. Dilkeshwar Pandey and all the faculty member of the Computer
Science and engineering department of ABES Institute of Technology for motivation and encouragement which helped
us to complete this review paper.
REFERENCES
[1] Garrison, G., Kim, S., Wakefield, R.L.: Success Factors for Deploying Cloud Computing. Commun. ACM. 55,
62–68 (2012).
[2] Herhalt, J., Cochrane, K.: Exploring the Cloud: A Global Study of Governments‘ Adoption of Cloud (2012).
[3] Sales force, ―CRM‖, http://www.salesforce.com/.
[4] Venters, W., Whitley, E.A.: A Critical Review of Cloud Computing: Researching Desires and Realities. J. Inf.
Technol. 27, 179–197 (2012).
[5] Yang, H., Tate, M.: A Descriptive Literature Review and Classification of Cloud Computing Research.
Commun. Assoc. Inf. Syst. 31 (2012).
[6] Marston, S., Li, Z., Bandyopadhyay, S., Zhang, J., Ghalsasi, A.: Cloud computing — The Business Perspective.
Decis. Support Syst. 51, 176–189 (2011).
ABOUT THE AUTHORS
1
Priyanshu Srivastava: He is a Scholar in the Computer Science and Engineering Department at ABES
Institute of Technology, Ghaziabad, since August‘2015. He is a member in Computer Society of India.
2
Prof. Rizwan Khan: He is the Assistant Professor in the Computer Science and Engineering Department at
ABES Institute of Technology, Ghaziabad.""",
]

## Choosing the best answer between pages

In [ ]:
def giveanswer(question, pdf,manuel_pdf):

  print("-----------------------")
  # In order to compare the answers, two initial value is created
  max_start_id = -50
  max_end_id = -100
  for page in pdf: 
    page = re.sub(' +', ' ', page) # in order to remove unncessary spaces in PDF
    tokenizer_out = tokenizer(question, page, truncation=True, padding=True)
    input_ids, attention_mask = tokenizer_out["input_ids"], tokenizer_out["attention_mask"]

    # Converting them to tensor because model works with them
    input_ids_tensor = nn.tensor([input_ids])
    attention_mask = nn.tensor([attention_mask])

    output = model(input_ids_tensor, attention_mask)
    start_id = nn.argmax(output[0])
    end_id = nn.argmax(output[1]) + 1

    # In order to compare results for each page's answer
    start_id_value = (output[0][0][start_id.item()].item())
    end_id_value = (output[1][0][end_id.item()-1].item())
    # In order to convert tokens to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start_id : end_id] , skip_special_tokens=True))
    # If the answer is empty, continue
    if answer == "":
      continue
    if (max_start_id + max_end_id) < (start_id_value + end_id_value ):
      max_start_id = start_id_value
      max_end_id = end_id_value
      final_start_id = start_id
      final_end_id = end_id
      final_answer = answer
  
  print(f"Question is {question}")
  print(f"Automatically converted PDF's answer is -> {final_answer}")

  # In order to compare the answers, two initial value is created
  max_start_id = -50
  max_end_id = -100
  for page in manuel_pdf: 
    page = re.sub(' +', ' ', page) # in order to remove unncessary spaces in PDF
    tokenizer_out = tokenizer(question, page, truncation=True, padding=True)
    input_ids, attention_mask = tokenizer_out["input_ids"], tokenizer_out["attention_mask"]

    # Converting them to tensor because model works with them
    input_ids_tensor = nn.tensor([input_ids])
    attention_mask = nn.tensor([attention_mask])

    output = model(input_ids_tensor, attention_mask)
    start_id = nn.argmax(output[0])
    end_id = nn.argmax(output[1]) + 1

    # In order to compare results for each page's answer
    start_id_value = (output[0][0][start_id.item()].item())
    end_id_value = (output[1][0][end_id.item()-1].item())
    # In order to convert tokens to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start_id : end_id] , skip_special_tokens=True))
    # If the answer is empty, continue
    if answer == "":
      continue
    if (max_start_id + max_end_id) < (start_id_value + end_id_value ):
      max_start_id = start_id_value
      max_end_id = end_id_value
      final_start_id = start_id
      final_end_id = end_id
      final_answer_manual = answer
    # In order to convert tokens to string
  print(f"Manually converted PDF's answer is -> {final_answer_manual}")
  print("-----------------------")

As can be seen from results, model works well on "what are" questions on both manuel and automatically converted PDF

In [ ]:
giveanswer("What is hybrid cloud?",pdf,manuel_pdf)
giveanswer("What are data centers?",pdf,manuel_pdf)

-----------------------
Question is What is hybrid cloud?
Automatically converted PDF's answer is -> the combination of public cloud and private cloud
Manually converted PDF's answer is -> public cloud and private cloud
-----------------------
-----------------------
Question is What are data centers?
Automatically converted PDF's answer is -> data centres are the compilation of servers
Manually converted PDF's answer is -> data centres are the compilation of servers
-----------------------


As can be seen from results, model especially automatically converted PDF cannot understand "who" question, manually converted PDF understands the question however it gives wrong answer.

In [ ]:
giveanswer("Who is the author of the paper?",pdf,manuel_pdf)

-----------------------
Question is Who is the author of the paper?
Automatically converted PDF's answer is -> priyanshu srivastava rizwan khan tata consultancy services limited abes institute of technology 1 publication 32 citations 61 publications 153 citations see profile see profile some of the authors of this publication are also working on these related projects : cloud computing view project machine learning view project all content following this page was uploaded by rizwan khan
Manually converted PDF's answer is -> mr. dilkeshwar pandey
-----------------------


Again, manually converted PDF works better.

In [ ]:
giveanswer("which service does enable to user can deploy their own software and coding?",pdf,manuel_pdf)

-----------------------
Question is which service does enable to user can deploy their own software and coding?
Automatically converted PDF's answer is -> amazon web services
Manually converted PDF's answer is -> paas
-----------------------
